In [3]:
import os
from dotenv import load_dotenv
load_dotenv()

import pandas as pd
import requests

In [2]:
df = pd.read_csv("./input/books.csv", error_bad_lines=False)


b'Skipping line 4012: expected 10 fields, saw 11\nSkipping line 5688: expected 10 fields, saw 11\nSkipping line 7056: expected 10 fields, saw 11\nSkipping line 10601: expected 10 fields, saw 11\nSkipping line 10668: expected 10 fields, saw 11\n'


In [3]:
df = df.drop(['isbn','language_code','text_reviews_count'], axis=1)
df = df.rename(columns = {'# num_pages':'num_pages'})
df.head(10)

,bookID,title,authors,average_rating,isbn13,num_pages,ratings_count
0,1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling-Mary GrandPré,4.56,9780439785969,652,1944099
1,2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling-Mary GrandPré,4.49,9780439358071,870,1996446
2,3,Harry Potter and the Sorcerer's Stone (Harry P...,J.K. Rowling-Mary GrandPré,4.47,9780439554930,320,5629932
3,4,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,4.41,9780439554893,352,6267
4,5,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling-Mary GrandPré,4.55,9780439655484,435,2149872
5,8,Harry Potter Boxed Set Books 1-5 (Harry Potte...,J.K. Rowling-Mary GrandPré,4.78,9780439682589,2690,38872
6,9,"Unauthorized Harry Potter Book Seven News: ""Ha...",W. Frederick Zimmerman,3.69,9780976540601,152,18
7,10,Harry Potter Collection (Harry Potter #1-6),J.K. Rowling,4.73,9780439827607,3342,27410
8,12,The Ultimate Hitchhiker's Guide: Five Complete...,Douglas Adams,4.38,9780517226957,815,3602
9,13,The Ultimate Hitchhiker's Guide to the Galaxy,Douglas Adams,4.38,9780345453747,815,240189


In [7]:
def githubRequestAuthorized(isbn):
    authToken = os.getenv("GOOGLE_BOOKS_API_TOKEN")
    if not authToken:
        raise ValueError("NECESITAS UN TOKEN")
    url = "https://www.googleapis.com/books/v1/volumes?q=isbn:{}&key={}".format(isbn,authToken)
    #print("Requesting authorized")
    res = requests.get(url)
    data = res.json()
    return data

In [8]:
def getGenre(isbn):
    url = "https://www.googleapis.com/books/v1/volumes?q=isbn:{}".format(isbn)
    print("Requesting url {}...".format(url))
    res = requests.get(url)
    data = res.json()
    print("done")
    
    return data
#data['ISBN:'+isbn]['details']['genres']


In [9]:
for i in range(len(df)) : 
    isbn = df.loc[i, "isbn13"]
    book = githubRequestAuthorized(isbn)
    try:
        df.loc[i, 'Genre'] = book['items'][0]['volumeInfo']['categories']
        
    except:
        df.loc[i, 'Genre'] = 'Others'


KeyboardInterrupt: 

In [1]:
df.head(100)

NameError: name 'df' is not defined

In [11]:
df.to_csv(r'./output/books_enriched.csv')

In [10]:
df = pd.read_csv("./output/books_enriched.csv")

In [11]:
df

,Unnamed: 0,bookID,title,authors,average_rating,isbn13,num_pages,ratings_count,Genre
0,0,1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling-Mary GrandPré,4.56,9780439785969,652,1944099,Juvenile Fiction
1,1,2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling-Mary GrandPré,4.49,9780439358071,870,1996446,Juvenile Fiction
2,2,3,Harry Potter and the Sorcerer's Stone (Harry P...,J.K. Rowling-Mary GrandPré,4.47,9780439554930,320,5629932,Juvenile Fiction
3,3,4,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,4.41,9780439554893,352,6267,Juvenile Fiction
4,4,5,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling-Mary GrandPré,4.55,9780439655484,435,2149872,Juvenile Fiction
...,...,...,...,...,...,...,...,...,...
13709,13709,47699,M Is for Magic,Neil Gaiman-Teddy Kristiansen,3.82,9780061186424,260,11317,NaN
13710,13710,47700,Black Orchid,Neil Gaiman-Dave McKean,3.72,9780930289553,160,8710,NaN
13711,13711,47701,InterWorld (InterWorld #1),Neil Gaiman-Michael Reaves,3.53,9780061238963,239,14334,NaN
13712,13712,47708,The Faeries' Oracle,Brian Froud-Jessica Macbeth,4.43,9780743201117,224,1550,NaN


In [12]:
dfgb = df.groupby(['Genre']).count()

In [13]:
dfgb.sort_values(['title'], ascending=[False]).head(10)

,Unnamed: 0,bookID,title,authors,average_rating,isbn13,num_pages,ratings_count
Genre,,,,,,,,
Others,826,826,826,826,826,826,826,826
Fiction,216,216,216,216,216,216,216,216
Drama,64,64,64,64,64,64,64,64
Biography & Autobiography,49,49,49,49,49,49,49,49
Literary Criticism,32,32,32,32,32,32,32,32
History,30,30,30,30,30,30,30,30
Juvenile Fiction,26,26,26,26,26,26,26,26
Business & Economics,22,22,22,22,22,22,22,22
Philosophy,19,19,19,19,19,19,19,19


In [19]:
selection = df.loc[df['Genre'] == 'Juvenile Fiction'].sort_values(['average_rating'], ascending=[False]).head(10)


In [20]:
finaldf = selection.drop(['Unnamed: 0','bookID','isbn13','ratings_count'], axis=1)


In [21]:
finaldf

,title,authors,average_rating,num_pages,Genre
5,Harry Potter Boxed Set Books 1-5 (Harry Potte...,J.K. Rowling-Mary GrandPré,4.78,2690,Juvenile Fiction
7,Harry Potter Collection (Harry Potter #1-6),J.K. Rowling,4.73,3342,Juvenile Fiction
0,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling-Mary GrandPré,4.56,652,Juvenile Fiction
4,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling-Mary GrandPré,4.55,435,Juvenile Fiction
1,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling-Mary GrandPré,4.49,870,Juvenile Fiction
2,Harry Potter and the Sorcerer's Stone (Harry P...,J.K. Rowling-Mary GrandPré,4.47,320,Juvenile Fiction
3,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,4.41,352,Juvenile Fiction
693,Harry Potter Schoolbooks Box Set: Two Classic ...,J.K. Rowling,4.40,240,Juvenile Fiction
176,The Phantom Tollbooth,Norton Juster-Jules Feiffer,4.21,256,Juvenile Fiction
786,Secret of the Peaceful Warrior,Dan Millman-Robert D. San Souci-T. Taylor Bruce,4.18,32,Juvenile Fiction
